<a href="https://colab.research.google.com/github/AmoghGorthi/Tic-Tac-Toe-using-MCTS/blob/main/TicTacToeusingMCTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import numpy as np
import random
import math
from collections import defaultdict


In [25]:
class TicTacToe:
    def __init__(self):
        self.board = np.zeros((3, 3), dtype=int)  # 3x3 Tic-Tac-Toe board
        self.current_player = 1  # Player 1 starts

    def get_valid_moves(self):
        return [(r, c) for r in range(3) for c in range(3) if self.board[r, c] == 0]

    def make_move(self, row, col):
        if self.board[row, col] == 0:
            self.board[row, col] = self.current_player
            self.current_player = 3 - self.current_player  # Switch player
            return True
        return False

    def is_winner(self, player):
        for row in range(3):
            if np.all(self.board[row, :] == player):
                return True
        for col in range(3):
            if np.all(self.board[:, col] == player):
                return True
        if np.all(np.diag(self.board) == player) or np.all(np.diag(np.fliplr(self.board)) == player):
            return True
        return False

    def is_draw(self):
        return len(self.get_valid_moves()) == 0 and not self.is_winner(1) and not self.is_winner(2)

    def game_over(self):
        return self.is_winner(1) or self.is_winner(2) or self.is_draw()

    def get_winner(self):
        if self.is_winner(1):
            return 1
        if self.is_winner(2):
            return 2
        return 0  # Draw or ongoing

    def copy(self):
        new_game = TicTacToe()
        new_game.board = np.copy(self.board)
        new_game.current_player = self.current_player
        return new_game

    def print_board(self):
        symbols = {0: ".", 1: "X", 2: "O"}
        for row in self.board:
            print(" ".join(symbols[cell] for cell in row))
        print()


In [26]:
class MCTSNode:
    def __init__(self, game, parent=None, move=None):
        self.game = game.copy()
        self.parent = parent
        self.move = move
        self.visits = 0
        self.wins = 0
        self.children = {}

    def is_fully_expanded(self):
        return len(self.children) == len(self.game.get_valid_moves())

    def best_child(self, exploration_weight=1.4):
        if not self.children:
            return None
        return max(self.children.values(),
                   key=lambda node: (node.wins / node.visits) + exploration_weight * math.sqrt(math.log(self.visits) / (1 + node.visits)))


In [28]:
import random

class MCTS:
    def __init__(self, simulations=1000):
        self.simulations = simulations  # Number of MCTS iterations

    def select(self, node):
        """Selection: Traverses the tree using UCT formula to find the best node."""
        while not node.game.game_over():
            if not node.is_fully_expanded():
                return self.expand(node)  # Expand if not fully explored
            node = node.best_child()  # Select the best child
        return node

    def expand(self, node):
        """Expansion: Adds a new child node by making a new move."""
        available_moves = node.game.get_valid_moves()
        for move in available_moves:
            if move not in node.children:
                new_game = node.game.copy()
                new_game.make_move(*move)
                child_node = MCTSNode(new_game, parent=node, move=move)
                node.children[move] = child_node
                return child_node
        return random.choice(list(node.children.values()))  # Fallback

    def simulate(self, node):
        """Simulation: Plays a random game from this state to get a result."""
        temp_game = node.game.copy()
        while not temp_game.game_over():
            move = random.choice(temp_game.get_valid_moves())  # Play random moves
            temp_game.make_move(*move)
        return temp_game.get_winner()

    def backpropagate(self, node, result):
        """Backpropagation: Updates the visit count and win statistics."""
        while node:
            node.visits += 1
            if result == node.game.current_player:
                node.wins += 1  # Reward wins
            node = node.parent  # Move up the tree

    def search(self, root_game):
        """Performs MCTS search and returns the best move."""
        root_node = MCTSNode(root_game)

        for _ in range(self.simulations):
            leaf_node = self.select(root_node)  # Step 1: Selection
            simulation_result = self.simulate(leaf_node)  # Step 3: Simulation
            self.backpropagate(leaf_node, simulation_result)  # Step 4: Backpropagation

        return root_node.best_child(exploration_weight=0).move  # Step 5: Best Move


In [29]:
def play_tic_tac_toe():
    game = TicTacToe()
    ai = MCTS(simulations=1000)  # You can adjust this for AI difficulty

    print("Tic-Tac-Toe MCTS AI")
    print("You are Player 1 (X) and AI is Player 2 (O)\n")

    # Display board positions to guide user
    print("Board Positions:")
    print(" 0 | 1 | 2 ")
    print("-----------")
    print(" 3 | 4 | 5 ")
    print("-----------")
    print(" 6 | 7 | 8 \n")

    while not game.game_over():
        game.print_board()

        if game.current_player == 1:  # Human's turn
            while True:
                try:
                    user_input = int(input("Enter a position (0-8): "))

                    if user_input in range(9):  # Ensure it's within range
                        row, col = divmod(user_input, 3)  # Convert 0-8 to row, col
                        if game.make_move(row, col):
                            break
                        else:
                            print("That spot is already taken! Try again.")
                    else:
                        print("Invalid input! Enter a number between 0 and 8.")
                except ValueError:
                    print("Invalid input! Enter a number between 0 and 8.")
        else:  # AI's turn
            print("AI is thinking...")
            row, col = ai.search(game)
            game.make_move(row, col)

    game.print_board()
    winner = game.get_winner()
    if winner == 1:
        print("You win! 🎉")
    elif winner == 2:
        print("AI wins! 🤖")
    else:
        print("It's a draw! 😐")


if __name__ == "__main__":
    play_tic_tac_toe()


Tic-Tac-Toe MCTS AI
You are Player 1 (X) and AI is Player 2 (O)

Board Positions:
 0 | 1 | 2 
-----------
 3 | 4 | 5 
-----------
 6 | 7 | 8 

. . .
. . .
. . .

Enter a position (0-8): 0
X . .
. . .
. . .

AI is thinking...
X . .
. . O
. . .

Enter a position (0-8): 2
X . X
. . O
. . .

AI is thinking...
X . X
. . O
. O .

Enter a position (0-8): 1
X X X
. . O
. O .

You win! 🎉
